# 트위터 검색 크롤러

트위터에서 특정 keyword, 기간의 결과물을 날짜별로 txt파일에 저장하는 코드 (using Chromedriver)

-> 개선해야될 부분
- 유저 단위로 나누기 (ID도 지우기)
- 좋아요, 리트윗 등 숫자 지우기
- 광고 거르고 싶다



### 참조
- [Python, Selenium 사용법](https://wordbe.tistory.com/entry/%ED%81%AC%EB%A1%A4%EB%A7%81-Python-Selenium-%EC%82%AC%EC%9A%A9%EB%B2%95) 
- [파이썬을 이용한 Twitter 크롤링](https://medium.com/@whj2013123218/%ED%8C%8C%EC%9D%B4%EC%8D%AC%EC%9D%84-%EC%9D%B4%EC%9A%A9%ED%95%9C-twitter-%ED%81%AC%EB%A1%A4%EB%A7%81-576f7b098daf)
- [인프런 - Twitter CSS 참고](https://www.inflearn.com/questions/42753)
- [CSS Selector를 사용한 크롤링](https://www.fun-coding.org/crawl_basic4.html)

이 블로그들을 기반으로 응용해 보았다! 

> selenium, BeautifulSoup, chromedriver 설치 후 사용

In [ ]:
! pip install selenium
! pip install bs4

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import datetime as dt

# 크롬드라이버 저장 경로
chromedriver = '<로컬 경로 입력>'

# 검색할 날짜 list로 설정 [month, day] 형태
# 예) match = [[10, 19], [10, 23], [10, 27]]
match = [[0, 0], [0, 0]]

# 검색 키워드 지정
keyword = "<키워드>"

# 확인할 기간
interval = <기간>

for m in match:
    # 크롬 드라이버 연결 (중요!)
    # browser을 새로 열 때마다 새로 연결해야한다! 
    browser = webdriver.Chrome(chromedriver)
    
    # 검색 기간(날짜) 설정
    startdate = dt.date(year=2020, month=m[0], day=m[1])
    untildate = startdate + dt.timedelta(days=1) # 하루하루의 결과를 출력하기 위한 중간변수
    enddate = startdate + dt.timedelta(days=interval)

    # 크롤링 결과 txt파일로 저장 <파일명_월_일.txt>
    # 날짜별 정렬을 위해 자리수를 2자리로 맞춰준다
    filename = "<파일명>_" + str('{0:0>2}'.format(m[0])) + '_' + str('{0:0>2}'.format(m[1])) + ".txt"
    file = open(filename, 'w')

    while not enddate == startdate:
        # url에서 until + <날짜> 는 결과에 포함되지 않는다.
        url='https://twitter.com/search?q='+keyword+'%20since%3A'+str(startdate)+'%20until%3A'+str(untildate)+'&amp;amp;amp;amp;amp;amp;lang=eg'

        # 크롬 실행
        browser.get(url)
        html = browser.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        # 로딩 기다림
        time.sleep(3)
        
        lastHeight = browser.execute_script("return document.body.scrollHeight")
        
        
        while True:

            tweet = soup.select('div#react-root main section article span')

            if len(tweet) > 12: # 12 필요없는 데이터로 취급하고 저장하지 않는다.
                for x in range(len(tweet)):
                    file.write(tweet[x].get_text())
                file.write('\n')

            # 스크롤 내리는 코드 (트위터 특성상 스크롤을 내려야 한다)
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1)

            newHeight = browser.execute_script("return document.body.scrollHeight")

            if newHeight != lastHeight:
                html = browser.page_source
                soup = BeautifulSoup(html, 'html.parser')
            else:
                startdate=untildate
                untildate += dt.timedelta(days=1)
                break
            lastHeight = newHeight
    
    file.close()
    print(m[0],  m[1]) # 완료된 날짜 출력

    browser.quit()
    time.sleep(2)
    
print('All Done')